<a href="https://colab.research.google.com/github/TugbaGoncu/YanginTespiti/blob/main/Copy_of_Yolov3_Egitim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  YOLOV3 NESNE TANIMA (OBJECT DETECTİON)
Bu notebookta nesne tanıma (object detection) için en hızlı modellerden biri olan yolov3 modeli ile nesne tanıma yapacağız.

1.ADIM = DARKNET KLONLAMA VE KURULUMU

In [ ]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 14621, done.
remote: Total 14621 (delta 0), reused 0 (delta 0), pack-reused 14621
Receiving objects: 100% (14621/14621), 13.21 MiB | 24.02 MiB/s, done.
Resolving deltas: 100% (9955/9955), done.


In [ ]:
#GPU ve opencv kullanabilmek için makefile dosyasında yapılması gereken değişiklikler.
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile

/content/darknet


In [ ]:
#Darknet Build
!make

mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:926:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1127:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
             ^~
./src/image_

## 2.ADIM = ÖNCEDEN EĞİTİLMİŞ YOLOV3 AĞIRLIKLARINI İNDİRME
YOLOV3'ün halihazırda 80 farklı sınıf barındıran 'coco' data setiyle eğitilmiş bir modeli mevcut . Bu eğitilien modelin ağırlıklarını indirerek istediğimiz fotoğraflar üzerinde tanımlamalar yaptırabiliriz. 

In [ ]:
!wget https://pjreddie.com/media/files/yolov3.weights

In [ ]:
#Yardımcı olacak fonksiyonlar
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()


# use this to download a file  
def download(path):
  from google.colab import files
  files.download(path)

## DARKNET VE YOLOV3 İLE İLK TESTİMİZİ YAPABİLİRİZ.

In [ ]:
# DARKNET DETECTİON FONKSİYONUNU ÇALIŞTIRALIM.
!./darknet detect cfg/yolov3.cfg yolov3.weights data/trafik.jpg

In [ ]:
%cd /content/darknet
download('predictions.jpg')

/content/darknet


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Önceden yazmış olduğumuz yardımcı fonksiyonumuz ile tanıma yapılmış fotoğrafı açalım.
imShow('predictions.jpg')

## Kendi Fotoğraflarımız Üzerinde Nesne Tanıma

In [ ]:
#Tanıma yapabilmek için darknet dizininde olmalıyız
#Fotoğrafımızı da content dizini içine atıyoruz
!./darknet detect cfg/yolov3.cfg yolov3.weights data/trafik.jpg
imShow('predictions.jpg')

In [ ]:
#Tanıma yapmış olduğumuz fotoğrafı indirebiliriz
       

## KENDİ NESNEMİZ İÇİN MODEL EĞİTİMİ
Şimdiye kadar yaptıklarımız daha önceden farklı nesneler için eğitilmiş modellerin ağırlıklarını kullanmaktı. 
Şimdi ise tamamen bizim nesne tanıma yapmak istediğimiz nesneler için eğitim yapacağız.

## ADIM 1: FOTOĞRAF TOPLAMA VE ETİKETLEME 
Üzerinde tanıma yapacağımız nesne için bir çok fotoğrafa ihtiyacımız olacak. Ne kadar fazla ve ne kadar farklı tip fotoğrafımız olursa o kadar doğru bir modelimiz olur. 

Fotoğraflarımızı 2 şekilde hazırlayabiliriz:

1)Google'ın sunmuş olduğu [Open İmages](https://storage.googleapis.com/openimages/web/index.html) sitesinde 600'ün üzerinde nesnenin etiketlenmiş halde binlerce fotoğrafı yer alıyor. Bu siteden istediğimiz kadar fotoğrafı indirip kullanabiliriz.(Open İmages kaynağından fotoğraf indirme için(https://www.youtube.com/watch?v=SjsF8_kDigw))

2)Tanıma yapacağımız nesneleri kendimiz fotoğraflayabilir veya google'dan bulduğumuz fotoğrafları etiketleyerek kullanabiliriz.(Kendi fotoğraflarımızı etiketlemek için(https://www.youtube.com/watch?v=H10L5BXdjdY&t=1s))

In [ ]:
#Öncelikle collabdan driveımıza bağlanmak için bir doğrulama yapmamız gerekiyor.
#Aşağıdaki kodu çalıştırarak çıkan linkte collab notebook'unu hangi hesabımız ile açtıysak o hesaba girerek gelen link ile doğrulama yapmalıyız.
%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

/content


In [ ]:
#Burada ise erişimi uzun olan drive dizini için kısa bir yol tanıma yapıyoruz
# "/content/gdrive/My\ Drive/" olan dizine /mydrive diyerek ulaşabiliriz.
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

## ADIM 2: HAZIRLAMIŞ OLDUĞUMUZ VERİ SETİNİ YÜKLEME
Hazırlamış olduğumuz veri setini google collab'e yüklemeliyiz.

Bunun için drive'ımızda bir yolov3 diye klasör oluşturalım.(bu klasörü daha sonra gerekli dosyaları bir arada toplamak için kullanacağız)

Veri setiniz bir klasör içine alıp zip haline getirerek drive'ımıza yükleyip oradan google collab içine aktarmak bize zaman kazandıracaktır.

In [ ]:
# oluşturduğumuz dosyayı kontrol edelim.
!ls /mydrive/yolov3

In [ ]:
# Yüklemiş olduğumuz .zip dosyasını collaba kopyalayalım
%cd darknet/
!cp /mydrive/yolov3/obj.zip ../

In [ ]:
# kopyaladığımız dosyayı zipten çıkaralım.
!unzip ../obj.zip -d data/

## ADIM 3: EĞİTİM İÇİN GEREKLİ DOSYALARI HAZIRLAYALIM

Bu adımda eğitim için gerekli olan .cfg file, obj.data, obj.names ve train.txt dosyalarını oluşturacağız.

*-Config Dosyası*

In [ ]:
#Öncelikle config dosyasını düzenleyelim.
#Darknet içindeki yolov3.cfg dosyasını drive'ımıza kopyalayalım ve adını değiştirelim.
!cp cfg/yolov3.cfg /mydrive/yolov3/yolov3_custom.cfg

Config dosyamızda yapmamız gereken değişiklikler:

(Burada verilen değerler bu değişkenlerin önerilen değerleridir.) 

1.   batch=1 ve subdivision = 1 satırlarını yorum satırı yapıyoruz.

2.   batch = 65 ve subdivision 16 satırlarını yorumdan çıkartıyoruz

3.   max_batches değerini (2000 * eğitilen sınıf sayısı) değerine eşitliyoruz.

4.   steps değerlerini (%80 of max_batches) , (%90 of max_batches) yapıyoruz.

5.   [yolo] başlığı altındaki classes değerlerini eğitim yaptığımız sınıf sayısı ile değiştiriyoruz. 

6.   filters değişkenlerini de (eğitim yapacağımız sınıf sayısı + 5 )*3 değerine eşitliyoruz 













In [ ]:
#Düzenlemiş olduğumuz config dosyasını geri yükleyelim
!cp /mydrive/yolov3/yolov3_custom.cfg ./cfg

*-obj.names ve obj.data*

yolov3 isimli klasörümüz içine obj.names isimli bir dosya oluşturalım ve dosyayı eğitim yapacağımız nesleriniz isimlerini yazıyoruz.

ÖRNEĞİN:
```
Guitar
Mobile Phone
```

Aynı klasör içinde obj.data isimle bir dosya oluşturarak içine eğitim yapacağımız nesne sayısını, eğitim yaparken kullanacağımız train.txt, text.txt ve obj.names isimli dosyaların adreslerini ve eğitim sonucu bulduğumuz ağırlıkları kaydedeceğimiz dizini yazıyoruz.

ÖRNEĞİN:
```
classes = 2
train = data/train.txt
valid = data/test.txt
names = data/obj.names
backup = /mydrive
```


In [ ]:
#Oluşturduğumuz obj.names ve obj.data dosyalarını google collaba aktaralım
!cp /mydrive/yolov3/obj.names ./data
!cp /mydrive/yolov3/obj.data  ./data

*-Train Dosyası*


generate_train.py



```
import os
image_files = []
os.chdir(os.path.join("data", "obj"))
for filename in os.listdir(os.getcwd()):
    if filename.endswith(".jpg"):
        image_files.append("data/obj/" + filename)
os.chdir("..")
with open("train.txt", "w") as outfile:
    for image in image_files:
        outfile.write(image)
        outfile.write("\n")
    outfile.close()
os.chdir("..")
```



In [ ]:
#Train dosyamızı google collaba kopyalayalım
!cp /mydrive/yolov3/generate_train.py ./

In [ ]:
#Şimdi bu python dosyasını çalıştırarak train için gerekli olayları bizim için yapmasını bekleyebiliriz.
!python generate_train.py

Eğer bu adıma kadar sorunsuz bir şekilde geldiysek train doğru bir şekilde geldiysek train.txt dosyamızı kontrol edelim. 

Dosyamıza çift tıklayıp açtığımızda her satırda 1 eğitim fotoğrafı olacak şekilde düzenlendiğiniz görebiliriz.

# ADIM 4: ÖNCEDEN EĞİTİLMİŞ CONVOLUTİONAL KATMANLARIN AĞIRLIKLARINI İNDİRME

Bu adımda önceden eğitilmiş yolov3 için kullanılmış deeplearning katmanları ağırlıklarını indiriyoruz. Bu adımı uygulamak zorunda değiliz ama eğitime bu ağırlıklarla başlamak eğittiğimiz modelin daha doğru çalışmasına ve eğitim süresini kısaltmaya yardımcı olacaktır.

In [ ]:
#Ağırlıkları alalım
!wget http://pjreddie.com/media/files/darknet53.conv.74

# ADIM 5: KENDİ NESNE TANIYICIMIZI EĞİTELİM

Gerekli tüm dosyalar hazır, eğitime başlayabiliriz.

Eğitimimiz uzun süreceği için google collab bizi serverdan atabilir. Bunun önüne geçmek için aktif olduğumuzu bir şekile bildirmeliyiz.


Bunun için de sayfanın üst tarafına sağ tıklayıp "ögeyi denetle" veya "incele" seçeneğiniz seçip, çıkan pencereden "console"'a tıklayıp açılan komut satırına aşağıdaki kodu ekleyip enter tuşuna basarsak bu kod bizim 10 dakikada bir connect butonuna basarak bizim aktif kalmamızı sağlayacaktır.

```
function ClickConnect(){
console.log("Working"); 
document.querySelector("colab-toolbar-button#connect").click() 
}
setInterval(ClickConnect,60000)
```

# EĞİTİM

Sıradaki komut ile eğitim başlayacaktır.

Eğitimimizin süresi veri setinizdeki fotoğraf sayısı, fotoğrafların kalitesi, eğitim yaptığınız nesne sayısı gibi faktörlere göre değişebilir. Modelimizin doğruluğu için loss değerimiz önemlidir. Loss değerimiz ne kadar düşük olursa modelimiz o kadar doğru çalışır. Modelimizi loss değeri azalmayı durdurana kadar çalıştırıp veri setimize göre mümkün olan en doğru modeli eğitebiliriz. 



In [ ]:
!./darknet detector train data/obj.data cfg/yolov3_custom.cfg  -dont_show

Modelimizi eğittikten sonra eğitim sırasında loss değerimizin nasıl değiştiğine dair bir grafik görebiliriz.

In [ ]:
imShow('chart.png')

Herhangi bir hata dolayısıyla eğittiğimiz modelimizin ağırlıklarını kaybetmemek için her 100 iterasyonda bir yeni ağırlıklarımızı drive'ımıza kaydediyoruz. Bu sayede hem ağırlıklarımızı kaybetmemiş oluyoruz hem de istediğimiz zaman modelimizi kaldığımız yerden eğitmeye devam edebiliyoruz.



In [ ]:
#Drive'ımıza kaydedilmiş ağırlıklar ile kaldığımız yerden eğitime devam edebiliriz.
!./darknet detector train data/obj.data cfg/yolov3_custom.cfg /mydrive/yolov3/yolov3_custom_last.weights -dont_show

# ADIM 6: EĞİTTİĞİMİZ MODELİMİZİ KULLANALIM


Eğitimimiz tamamlandı, şimdi istediğimiz fotoğraflar üzerinde tanıma yapabiliriz.

In [ ]:
# test için kendi config dosyamızı set edelim.
%cd cfg
!sed -i 's/batch=64/batch=1/' yolov3_custom.cfg
!sed -i 's/subdivisions=16/subdivisions=1/' yolov3_custom.cfg
%cd ..


In [ ]:
#Drive'ımıza yüklediğimiz fotoğraflar üzerinde test edebiliriz.
#Thresh değeri belirlediğimiz değerden yüksek olasılıklı tahminleri gösterir.
!./darknet detector test data/obj.data cfg/yolov3_custom.cfg /mydrive/yolov3/yolov3_custom_last.weights /mydrive/yolov3/fotograflar/gitar2.jpg -thresh 0.1
imShow('predictions.jpg')

In [ ]:
%cd /content/darknet
download('predictions.jpg')